Jupyter JavaScript Kernel Experiments
=====================================

As user code is executed in a WebWorker, **postMessage()** will be called to return any exexcution outputs that need to be displayed in the front-end. The **postMessage()** function can be redefined to allow the messages to be logged.

In [1]:
var message_log = "";
var pm = postMessage; // Keep reference to original postMessage() function.
var start_time  = Date.now();

function getTime() {
    return Date.now() - start_time;
}

// Redefine the postMessage() function and log the message objects.
postMessage = function(obj) {
    message_log += getTime() + ": " + JSON.stringify(obj) + "\n";
    pm(obj);
}

getTime();

Dumping the logged messages shows two types of message being sent to the front-end:
* [execute_result](https://jupyter-client.readthedocs.io/en/stable/messaging.html#id6)
* RAW

In [2]:
function dumpLog() {
    console.log(message_log);
    message_log="";
}

dumpLog();

0: {"bundle":{"data":{"text/plain":0},"metadata":{},"execution_count":1},"type":"execute_result"}
0: {"type":"RAW","value":{"status":"ok","user_expressions":{}},"id":"1d6a1d4f3d9dd-116a49065d4227-987de72767df3-18068701d63801"}



Delaying Notebook Execution
---------------------------

If the **execute_result** and **RAW** messages are not send to the front-end it will stall. The code below queues the messages if the **pause()** function is called and posts them after a specifed time. The **setTimeout()** function makes an asyncronous callback which is used to send the queued messages.

In [3]:
var messages = [];
var pause_cnt = 0;

pause = function(ms) {
    pause_cnt += 1;
    setTimeout(function() {
        resume();
    }, ms);
};

postMessage = function(obj) {
    message_log += getTime() + ": " + JSON.stringify(obj) + "\n";
    if (pause_cnt) messages.push(obj);
    else pm(obj);
};

resume = function() {
    pause_cnt -= 1;
    if (pause_cnt <= 0) {
        pause_cnt = 0;
        while (messages.length > 0) {
            var obj = messages.shift();
            pm(obj);
        }
    }
};

pause(2000);

In [4]:
pause(4000);

In [5]:
pause(2000);

Note that **dumpLog()** will not dump the **execute_result** and **RAW** from this execution as their respective **postMessage()** calls are not made until after the cell code is executed.

In [6]:
dumpLog();

7: {"bundle":{"data":{},"metadata":{},"execution_count":2},"type":"execute_result"}
7: {"type":"RAW","value":{"status":"ok","user_expressions":{}},"id":"7eedeecdf1eb1-a3968aba27d65-431fdb295ee9-67d3b180e92a9"}
27: {"bundle":{"data":{},"metadata":{},"execution_count":3},"type":"execute_result"}
27: {"type":"RAW","value":{"status":"ok","user_expressions":{}},"id":"1c51c1f4dfcc51-120817d049c059-8f326cd55ef81-c9a616b8dbd3b"}
2039: {"bundle":{"data":{},"metadata":{},"execution_count":4},"type":"execute_result"}
2039: {"type":"RAW","value":{"status":"ok","user_expressions":{}},"id":"a4dd44f76327d-23b8009d86c87-adc604554606f-156cc93b165a0b"}
6046: {"bundle":{"data":{},"metadata":{},"execution_count":5},"type":"execute_result"}
6046: {"type":"RAW","value":{"status":"ok","user_expressions":{}},"id":"bae02683d4995-12951998afbb4f-1252f756d3c07d-9662f93438273"}



In [7]:
dumpLog();

8055: {"bundle":{"data":{},"metadata":{},"execution_count":6},"type":"execute_result"}
8056: {"type":"RAW","value":{"status":"ok","user_expressions":{}},"id":"1a2456c221073d-c9bdd3fb35b91-1010cd0d5e9b57-b91bb6e354a39"}

